# Basic Agent Auth — 3 Scenarios Demo

This notebook runs the three authentication scenarios in `MICROSOFT-ENTRA-AGENT-ID/examples/basic-agent-auth`:

1. **Autonomous (Client Credentials)** — service principal + client secret
2. **Managed Identity / DefaultAzureCredential** — managed identity in Azure, CLI fallback locally
3. **Interactive (User Context)** — device code / browser sign-in (delegated)

Prereqs:
- Select the workspace `.venv` as the notebook kernel
- Ensure `.env` exists in this folder with the required values

## Microsoft Agent Identity Platform — Concepts

**The platform provides three core identity constructs:**

1. **Agent Identity Blueprint** — template/parent for managing multiple agent identities
2. **Agent Identity** — primary account used by an AI agent (no password; authenticates via tokens)
3. **Agent User** — secondary user-object account for systems requiring user principals

**Two access modes:**

- **Application-only (autonomous)** — agent acts using its own identity
  - Uses OAuth 2.0 client credentials flow
  - Our `autonomous_agent.py` demo
  
- **Delegated (interactive)** — agent acts on behalf of a signed-in user
  - Uses OAuth 2.0 + OIDC with user consent
  - Our `interactive_agent.py` demo

**Platform components:**

- Authentication Service: OAuth 2.0 / OIDC compliant token issuance
- SDKs: Microsoft Identity Web (.NET), Microsoft Entra SDK for Agent ID (containerized)
- Agent Registry: Centralized discovery and governance (Entra admin center)

**References:**

- [Microsoft Agent Identity Platform for Developers](https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/what-is-agent-id-platform)
- [Tokens in Agent Identity Platform](https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/agent-tokens)

In [1]:
import sys
import subprocess
from pathlib import Path

HERE = Path.cwd()
candidates = [
    HERE,
    HERE / "MICROSOFT-ENTRA-AGENT-ID" / "examples" / "basic-agent-auth",
]

BASE_DIR = next((p for p in candidates if (p / "autonomous_agent.py").exists()), None)
if BASE_DIR is None:
    raise FileNotFoundError(
        "Could not find basic-agent-auth scripts. \n"
        "Open this notebook from the examples folder, or set BASE_DIR manually."
    )

print("Python executable:", sys.executable)
print("Kernel CWD:", HERE)
print("Scripts dir:", BASE_DIR)

def run_script(script_name: str, *args: str) -> None:
    cmd = [sys.executable, script_name, *args]
    print("\n=== Running ===")
    print("Command: " + " ".join(cmd))
    print("CWD:     " + str(BASE_DIR))
    print("==============\n")
    result = subprocess.run(cmd, cwd=str(BASE_DIR), text=True)
    if result.returncode != 0:
        raise RuntimeError(f"{script_name} failed with exit code {result.returncode}")

Python executable: /Users/arturoquiroga/GITHUB/agent-framework-public/.venv/bin/python
Kernel CWD: /Users/arturoquiroga/GITHUB/agent-framework-public/MICROSOFT-ENTRA-AGENT-ID/examples/basic-agent-auth
Scripts dir: /Users/arturoquiroga/GITHUB/agent-framework-public/MICROSOFT-ENTRA-AGENT-ID/examples/basic-agent-auth


## 1) Autonomous (Client Credentials)
Runs `autonomous_agent.py` using `TENANT_ID`, `AGENT_CLIENT_ID`, and `AGENT_CLIENT_SECRET`.

In [2]:
run_script("autonomous_agent.py")


=== Running ===
Command: /Users/arturoquiroga/GITHUB/agent-framework-public/.venv/bin/python autonomous_agent.py
CWD:     /Users/arturoquiroga/GITHUB/agent-framework-public/MICROSOFT-ENTRA-AGENT-ID/examples/basic-agent-auth


Autonomous Agent (Client Credentials)
Agent ID: 2c9ecb92-2756-4983-a4c6-2884d8ba3fa1


DEMO 1: Azure Management Token
✓ Token acquired for: 2c9ecb92-2756-4983-a4c6-2884d8ba3fa1
  Expires: 2026-01-19 19:33:37

DEMO 2: Azure Storage Access
Storage account: aqmlwork0018580440867
Container: agent-test
✓ Container exists: agent-test
✓ Found 0 blobs

DEMO 3: Token Refresh and Caching
Acquiring first token...
✓ Token acquired for: 2c9ecb92-2756-4983-a4c6-2884d8ba3fa1
  Expires: 2026-01-19 19:33:36
Acquiring second token (should use cache)...
✓ Token acquired for: 2c9ecb92-2756-4983-a4c6-2884d8ba3fa1
  Expires: 2026-01-19 19:33:36
✓ Token was reused from cache
✓ Token is still valid

ALL DEMOS COMPLETED SUCCESSFULLY



### (Optional) Inspect Autonomous Token Claims

The autonomous agent receives an **application-only** token. Key claims to look for:

- `idtyp: app` — identifies this as an application token (not user)
- `appid` — the agent's client ID
- `aud` — the audience (which API this token is for)
- `roles` or `scp` — permissions granted to the agent

Agent-specific claims (if present):
- `xms_act_fct` — actor facet (who's performing the action)
- `xms_sub_fct` — subject facet (for whom the action is performed)

In [3]:
import os
import json
import base64
from dotenv import load_dotenv
from azure.identity import ClientSecretCredential

load_dotenv(BASE_DIR / ".env")

tenant_id = os.getenv("TENANT_ID")
client_id = os.getenv("AGENT_CLIENT_ID")
client_secret = os.getenv("AGENT_CLIENT_SECRET")

if all([tenant_id, client_id, client_secret]):
    cred = ClientSecretCredential(tenant_id, client_id, client_secret)
    token_result = cred.get_token("https://management.azure.com/.default")
    
    # Decode JWT (split on '.', base64 decode the payload)
    parts = token_result.token.split('.')
    if len(parts) == 3:
        # Add padding if needed
        payload = parts[1]
        payload += '=' * (4 - len(payload) % 4)
        claims = json.loads(base64.urlsafe_b64decode(payload))
        
        print("=== Key Token Claims (Application-Only) ===")
        print(f"idtyp:  {claims.get('idtyp', 'N/A')}")
        print(f"appid:  {claims.get('appid', 'N/A')}")
        print(f"aud:    {claims.get('aud', 'N/A')}")
        print(f"roles:  {claims.get('roles', 'N/A')}")
        print(f"scp:    {claims.get('scp', 'N/A')}")
        
        # Agent-specific claims (preview)
        if 'xms_act_fct' in claims:
            print(f"xms_act_fct: {claims['xms_act_fct']}")
        if 'xms_sub_fct' in claims:
            print(f"xms_sub_fct: {claims['xms_sub_fct']}")
        if 'xms_idrel' in claims:
            print(f"xms_idrel: {claims['xms_idrel']}")
    else:
        print("Could not decode token")
else:
    print("Skipping token inspection (missing credentials)")

=== Key Token Claims (Application-Only) ===
idtyp:  app
appid:  2c9ecb92-2756-4983-a4c6-2884d8ba3fa1
aud:    https://management.azure.com
roles:  N/A
scp:    N/A
xms_act_fct: 9 3
xms_sub_fct: 9 3
xms_idrel: 7 8


## 2) Managed Identity / DefaultAzureCredential
Runs `managed_identity_agent.py`.
- In Azure: uses Managed Identity
- Locally: falls back to `DefaultAzureCredential` (often Azure CLI)

In [4]:
run_script("managed_identity_agent.py")


=== Running ===
Command: /Users/arturoquiroga/GITHUB/agent-framework-public/.venv/bin/python managed_identity_agent.py
CWD:     /Users/arturoquiroga/GITHUB/agent-framework-public/MICROSOFT-ENTRA-AGENT-ID/examples/basic-agent-auth



2026-01-19 18:33:40,840 - __main__ - INFO - === Managed Identity Agent Demo ===

2026-01-19 18:33:40,840 - __main__ - INFO - ⚠ Not running in Azure environment
2026-01-19 18:33:40,840 - __main__ - INFO - Falling back to DefaultAzureCredential
2026-01-19 18:33:40,840 - __main__ - INFO - 
--- DefaultAzureCredential Demo ---
2026-01-19 18:33:40,851 - __main__ - INFO - DefaultAzureCredential agent initialized
2026-01-19 18:33:40,851 - __main__ - INFO - Will try: Environment → Managed Identity → CLI → ...
2026-01-19 18:33:45,375 - __main__ - INFO - ✓ Token acquired via DefaultAzureCredential
2026-01-19 18:33:47,381 - __main__ - INFO -   Likely method: Azure CLI or VS Code
2026-01-19 18:33:47,382 - __main__ - INFO - ✓ Token acquired (length: 2331)
2026-01-19 18:33:47,383 - __main__ - INFO - 
=== How to Enable Managed Identity ===
2026-01-19 18:33:47,383 - __main__ - INFO - 
For Azure VM:
2026-01-19 18:33:47,383 - __main__ - INFO -   az vm identity assign --name <vm-name> --resource-group <rg

## 3) Interactive (User Context)
Runs `interactive_agent.py` and forces device-code flow (works in terminal, SSH, and notebooks).

If prompted:
1. Open `https://microsoft.com/devicelogin`
2. Enter the code
3. Complete sign-in

In [5]:
run_script("interactive_agent.py", "--device-code")


=== Running ===
Command: /Users/arturoquiroga/GITHUB/agent-framework-public/.venv/bin/python interactive_agent.py --device-code
CWD:     /Users/arturoquiroga/GITHUB/agent-framework-public/MICROSOFT-ENTRA-AGENT-ID/examples/basic-agent-auth



2026-01-19 18:33:48,443 - __main__ - INFO - === Interactive Agent Demo ===

2026-01-19 18:33:48,443 - __main__ - INFO - Detected headless environment, using device code flow
2026-01-19 18:33:48,443 - __main__ - INFO - Using device code flow...
2026-01-19 18:33:48,449 - __main__ - INFO - Interactive agent initialized: c82b8fb8-4cd4-4512-8faa-82ef1671ee7e
2026-01-19 18:33:48,449 - __main__ - INFO - 
--- Demo 1: User Token Acquisition ---
2026-01-19 18:33:48,449 - __main__ - INFO - Requesting scopes: ['https://graph.microsoft.com/User.Read']
2026-01-19 18:33:48,859 - __main__ - INFO - 
2026-01-19 18:33:48,859 - __main__ - INFO - AUTHENTICATION REQUIRED
2026-01-19 18:33:48,859 - __main__ - INFO - ============================================================
2026-01-19 18:33:48,859 - __main__ - INFO - 1. Open: https://microsoft.com/devicelogin
2026-01-19 18:33:48,859 - __main__ - INFO - 2. Enter code: FL94F5S2A
2026-01-19 18:33:48,859 - __main__ - INFO - 3. Sign in and consent
2026-01-19 18:

## (Optional) LLM Variants
If you want the same three scenarios but including Azure OpenAI calls, run the `_llm.py` scripts below.

In [7]:
# Uncomment any you want to run:
run_script("autonomous_agent_llm.py")
# run_script("managed_identity_agent_llm.py")
# run_script("interactive_agent_llm.py", "--device-code")


=== Running ===
Command: /Users/arturoquiroga/GITHUB/agent-framework-public/.venv/bin/python autonomous_agent_llm.py
CWD:     /Users/arturoquiroga/GITHUB/agent-framework-public/MICROSOFT-ENTRA-AGENT-ID/examples/basic-agent-auth


Autonomous AI Agent with LLM
Agent ID: 2c9ecb92-2756-4983-a4c6-2884d8ba3fa1
Azure OpenAI: https://r2d2-foundry-001.openai.azure.com/
Model: gpt-4.1


DEMO 1: Agent Identity Verification
✓ Agent ID: 2c9ecb92-2756-4983-a4c6-2884d8ba3fa1
✓ Token expires: 2026-01-19 19:36:08
✓ Confirmed as agent identity (not user)

DEMO 2: Simple LLM Interaction
Query: What is Microsoft Entra Agent ID?
  Model: gpt-4.1-2025-04-14
  Tokens: 487

Response:
**Microsoft Entra Agent ID** is a cloud-based service that enables secure, centralized management of authentication agents within Microsoft Entra (formerly Azure Active Directory). It provides unique identity and lifecycle management for agents that interact with Entra services, such as:

- **Microsoft Entra Connect** (for hybri

---

## Platform Integration Notes

**Agent Registry & Discovery**

The Microsoft agent identity platform includes an **agent registry** (centralized repository) where agents register metadata for discovery and governance. While our demos focus on authentication/authorization mechanics, production agents would:

- Register in the Entra admin center
- Publish capabilities, tasks, and protocols (e.g., MCP, A2A)
- Enable agent-to-agent discovery across the organization

**Microsoft Entra SDK for Agent ID**

For production deployments, consider the **containerized SDK** that offloads:
- Token acquisition and validation
- Downstream API calls
- Protocol handling

Your application communicates with the SDK over HTTP, keeping identity logic isolated and manageable.

**Integration with Security Services**

The platform integrates with:
- Conditional Access (adaptive policies based on agent risk)
- Identity Protection (risk detection for agents)
- Identity Governance (lifecycle management)
- Global Secure Access (network-level controls)

See [Microsoft Entra Agent ID](https://learn.microsoft.com/en-us/entra/agent-id/identity-professional/microsoft-entra-agent-identities-for-ai-agents) for details.

---